In [1]:
import pandas as pd
import numpy as np

boxes = pd.read_csv('boxes.csv')

prob_1 = pd.read_csv("problem 1.csv")
sample_1 = pd.read_csv("sample submission 1.csv")

prob_3 = pd.read_csv("problem 3.csv")
sample_3 = pd.read_csv("sample submission 3.csv")

purchase = pd.read_csv('purchase.csv').dropna().reset_index(drop=True)
purchase = purchase[purchase['BOX_ID'] <= 290].reset_index(drop=True)

df = purchase.copy()
df.head()

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT
0,1/2/2019,2CED678A247,12.0,1.0
1,1/2/2019,2BF58D91BA1,12.0,1.0
2,1/2/2019,2C15B86534E,99.0,1.0
3,1/2/2019,2C32D9A859A,6.0,1.0
4,1/2/2019,2C7A55404D1,4.0,1.0


# Data Preaparation

In [2]:
df = pd.merge(df, boxes, on='BOX_ID', how='left')
df['PURCHASE_DATE'] = pd.to_datetime(purchase['PURCHASE_DATE'], dayfirst=True)

df['QUALITY'] = df['QUALITY'].map({'Premium':1, 'Standard':0})
df['DELIVERY_OPTION'] = df['DELIVERY_OPTION'].map({'Home Delivery - Digital Payment':2,
                                                   'Home Delivery - CoD':1,
                                                   'Delivery from Collection Point':0})


df['MEAT'] = df['MEAT'] * df['BOX_COUNT']
df['MILK'] = df['MILK'] * df['BOX_COUNT']

df['PURCHASE_MONTH'] = df['PURCHASE_DATE'].dt.month
df['PURCHASE_DAY'] = df['PURCHASE_DATE'].dt.day

df = df.sort_values(['MAGIC_KEY', 'PURCHASE_DATE']).reset_index(drop=True)
earliest_date = pd.to_datetime('2018-10-01')
df['LAST_PURCHASE'] = df.groupby('MAGIC_KEY')['PURCHASE_DATE'].diff().fillna(df['PURCHASE_DATE'] - earliest_date)
df['LAST_PURCHASE'] = df['LAST_PURCHASE'].dt.days

df = df[['PURCHASE_DATE', 'MAGIC_KEY', 'PURCHASE_MONTH', 'PURCHASE_DAY', 'MILK', 'MEAT', 'LAST_PURCHASE', 'QUALITY', 'DELIVERY_OPTION']]

df['freq_all'] = df['MAGIC_KEY'].map(df['MAGIC_KEY'].value_counts())
df['freq_monthly'] = df.groupby(['MAGIC_KEY', 'PURCHASE_MONTH'])['MAGIC_KEY'].transform('count')
df['monthly_freq_ratio'] = df['freq_monthly'] / df['freq_all']
df.drop(['freq_all', 'freq_monthly'], axis=1, inplace=True)

df.head()

,PURCHASE_DATE,MAGIC_KEY,PURCHASE_MONTH,PURCHASE_DAY,MILK,MEAT,LAST_PURCHASE,QUALITY,DELIVERY_OPTION,monthly_freq_ratio
0,2018-11-01,249670911D8,11,1,0.0,2.4,31,0,0,0.5
1,2019-02-02,249670911D8,2,2,8.5,0.0,93,0,0,0.5
2,2018-10-12,249751FC4DD,10,12,10.0,1.8,11,0,0,1.0
3,2018-10-19,24978027606,10,19,0.0,2.9,18,1,1,1.0
4,2018-11-30,24979164422,11,30,0.0,2.5,60,0,0,1.0


# Labeling Data for Problem 1

In [3]:
# Filter the DataFrame based on PURCHASE_MONTH and reset index
oct_train = df[df['PURCHASE_MONTH'] == 10].reset_index(drop=True)
nov_train = df[df['PURCHASE_MONTH'] == 11].reset_index(drop=True)
dec_train = df[df['PURCHASE_MONTH'] == 12].reset_index(drop=True)
jan_train = df[df['PURCHASE_MONTH'] == 1].reset_index(drop=True)

# Find unique MAGIC_KEY values for each month
oct_keys = set(oct_train['MAGIC_KEY'])
nov_keys = set(nov_train['MAGIC_KEY'])
dec_keys = set(dec_train['MAGIC_KEY'])
jan_keys = set(jan_train['MAGIC_KEY'])

# Filter next month's DataFrame for the first 15 days
nov_first_15_days = df[(df['PURCHASE_MONTH'] == 11) & (df['PURCHASE_DAY'] <= 15)]
dec_first_15_days = df[(df['PURCHASE_MONTH'] == 12) & (df['PURCHASE_DAY'] <= 15)]
jan_first_15_days = df[(df['PURCHASE_MONTH'] == 1) & (df['PURCHASE_DAY'] <= 15)]
feb_first_15_days = df[(df['PURCHASE_MONTH'] == 2) & (df['PURCHASE_DAY'] <= 15)]

# Check if MAGIC_KEY values from current month are present in the next month's first 15 days
oct_train['target_BOOL'] = oct_train['MAGIC_KEY'].isin(nov_first_15_days['MAGIC_KEY']).astype(bool)
nov_train['target_BOOL'] = nov_train['MAGIC_KEY'].isin(dec_first_15_days['MAGIC_KEY']).astype(bool)
dec_train['target_BOOL'] = dec_train['MAGIC_KEY'].isin(jan_first_15_days['MAGIC_KEY']).astype(bool)
jan_train['target_BOOL'] = jan_train['MAGIC_KEY'].isin(feb_first_15_days['MAGIC_KEY']).astype(bool)

# Labeling for Problem 3

In [4]:
# November
nov_meat = nov_first_15_days.groupby('MAGIC_KEY')['MEAT'].sum()
oct_train = oct_train.merge(nov_meat.rename('target_MEAT'), on='MAGIC_KEY', how='left')

# December
dec_meat = dec_first_15_days.groupby('MAGIC_KEY')['MEAT'].sum()
nov_train = nov_train.merge(dec_meat.rename('target_MEAT'), on='MAGIC_KEY', how='left')

# January
jan_meat = jan_first_15_days.groupby('MAGIC_KEY')['MEAT'].sum()
dec_train = dec_train.merge(jan_meat.rename('target_MEAT'), on='MAGIC_KEY', how='left')

# February
feb_meat = feb_first_15_days.groupby('MAGIC_KEY')['MEAT'].sum()
jan_train = jan_train.merge(feb_meat.rename('target_MEAT'), on='MAGIC_KEY', how='left')

# Train_Test

In [5]:
data = pd.concat([oct_train, nov_train, dec_train, jan_train], ignore_index=True)
data = data.fillna(0)

# Comment the line below to train all the data
bool_data = data[data['MAGIC_KEY'].isin(prob_1['MAGIC_KEY'])].reset_index(drop=True)
bool_train = bool_data[['PURCHASE_DATE', 'MAGIC_KEY', 'PURCHASE_DAY', 'MILK', 'MEAT', 'LAST_PURCHASE', 'monthly_freq_ratio', 'QUALITY', 'DELIVERY_OPTION']]
bool_target = bool_data['target_BOOL']

meat_data = data[data['target_BOOL']].reset_index(drop=True)
meat_train = meat_data[['PURCHASE_DATE', 'MAGIC_KEY', 'PURCHASE_DAY', 'MILK', 'MEAT', 'LAST_PURCHASE', 'monthly_freq_ratio', 'QUALITY', 'DELIVERY_OPTION']]
meat_target = meat_data['target_MEAT']

test = df.sort_values(by=['MAGIC_KEY', 'PURCHASE_DATE'], ascending=[True, False]).reset_index(drop=True)
test = test.drop_duplicates(subset='MAGIC_KEY', keep='first').reset_index(drop=True)
test = test[['PURCHASE_DATE', 'MAGIC_KEY', 'PURCHASE_DAY', 'MILK', 'MEAT', 'LAST_PURCHASE', 'monthly_freq_ratio', 'QUALITY', 'DELIVERY_OPTION']]

test_bool = test[test['MAGIC_KEY'].isin(prob_1['MAGIC_KEY'])].reset_index(drop=True)
test_meat = test[test['MAGIC_KEY'].isin(prob_3['MAGIC_KEY'])].reset_index(drop=True)

In [6]:
pd.DataFrame(meat_target).describe()

,target_MEAT
count,454821.000000
mean,2.001687
std,1.119292
min,0.000000
25%,1.800000
50%,2.200000
75%,2.400000
max,23.200000


In [7]:
pd.DataFrame(bool_target).value_counts()

target_BOOL
False          91373
True           75792
Name: count, dtype: int64

# Modeling 

### Model for problem 01

In [8]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from xgboost import XGBClassifier

model = XGBClassifier()

cross_validate(model, bool_train.drop(['PURCHASE_DATE', 'MAGIC_KEY'], axis=1), bool_target,
               cv=StratifiedKFold(10),
               scoring='accuracy',
               return_train_score=True)

{'fit_time': array([1.2797842 , 0.36045146, 0.36047649, 0.37748051, 0.41483021,
        0.39585328, 0.40180516, 0.54002023, 0.34955239, 0.35195303]),
 'score_time': array([0.        , 0.0117135 , 0.        , 0.00943112, 0.0116353 ,
        0.        , 0.01224208, 0.        , 0.01167512, 0.01325631]),
 'test_score': array([0.76867859, 0.77585691, 0.78064246, 0.74989532, 0.76945624,
        0.79690117, 0.78433836, 0.79283321, 0.79797799, 0.79612347]),
 'train_score': array([0.79920637, 0.79817611, 0.79939913, 0.80170557, 0.80010369,
        0.79783847, 0.79894183, 0.79765236, 0.79825057, 0.7976723 ])}

### Model for problem 03

In [9]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from xgboost import XGBRegressor

model = XGBRegressor()

cross_validate(model, meat_train.drop(['PURCHASE_DATE', 'MAGIC_KEY'], axis=1), meat_target,
               cv=10,
               scoring='neg_root_mean_squared_error',
               return_train_score=True)

{'fit_time': array([1.38785052, 1.29781961, 1.32519603, 1.24574614, 1.30374932,
        1.29977751, 1.25439191, 1.26111746, 1.25607777, 1.22512031]),
 'score_time': array([0.01308727, 0.01550436, 0.03131533, 0.02378082, 0.01562476,
        0.01742554, 0.02487826, 0.00672293, 0.01101851, 0.01570272]),
 'test_score': array([-0.71309611, -0.55895243, -0.60397401, -0.5556972 , -0.58893042,
        -0.60448864, -0.60867219, -0.6227897 , -0.62964997, -0.61000028]),
 'train_score': array([-0.56355824, -0.57282706, -0.56863283, -0.57346245, -0.5701901 ,
        -0.57131399, -0.57058326, -0.56932041, -0.56749277, -0.56819224])}

# Predictions and Submissions

In [10]:
from xgboost import XGBClassifier
clf = XGBClassifier()

clf.fit(bool_train.drop(['PURCHASE_DATE', 'MAGIC_KEY'], axis=1), bool_target)

p1 = clf.predict(test_bool.drop(['PURCHASE_DATE', 'MAGIC_KEY'], axis=1))
test_bool.loc[:, 'PURCHASE'] = p1

sub = pd.merge(prob_1, test_bool, on='MAGIC_KEY', how='left')
sub = sub[['MAGIC_KEY', 'PURCHASE']]

sub['PURCHASE'] = sub['PURCHASE'].map({1:'Y', 0:'N'})
sub.to_csv('month_based_P1.csv', index=None)

sub

,MAGIC_KEY,PURCHASE
0,28D5BB06356,N
1,293BEAB4E98,Y
2,2962EE8065C,N
3,2957BE29EA9,N
4,28E351A0745,N
...,...,...
58684,28FB7C09776,N
58685,28E0E3B69BF,N
58686,28D343103A7,N
58687,290B1D6D5CB,N


In [11]:
sub['PURCHASE'].value_counts()

PURCHASE
N    36195
Y    22494
Name: count, dtype: int64

In [12]:
sample_1.shape, sub.shape

((58689, 2), (58689, 2))

In [13]:
from xgboost import XGBRegressor
regr = XGBRegressor()

regr.fit(meat_train.drop(['PURCHASE_DATE', 'MAGIC_KEY'], axis=1), meat_target)

p3 = regr.predict(test_meat.drop(['PURCHASE_DATE', 'MAGIC_KEY'], axis=1))
test_meat.loc[:,'preds'] = p3 + abs(min(p3))

sub = pd.merge(prob_3, test_meat, on='MAGIC_KEY', how='left')
sub = sub[['MAGIC_KEY', 'preds']]
sub.rename(columns={'preds': 'MEAT'}, inplace=True)

sub.fillna(0, inplace=True)
sub.to_csv('month_based_P3.csv', index=None)

sub

,MAGIC_KEY,MEAT
0,2BCFE9C06A7,7.016002
1,2C2A872B5A2,4.335819
2,2C6A897671B,4.356737
3,2C6F1287F53,10.742587
4,2C658198CC9,4.338244
...,...,...
5374,2C0804EFE49,3.251038
5375,2C080B48630,2.011938
5376,2C08243C58E,2.802018
5377,2C082C78575,3.072096


In [14]:
sub['MEAT'].describe()

count    5379.000000
mean        3.586764
std         1.239417
min         0.000000
25%         2.870429
50%         3.355543
75%         4.171070
max        21.026117
Name: MEAT, dtype: float64

In [15]:
sample_3.shape, sub.shape

((5379, 2), (5379, 2))